In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from epydemix.population import load_epydemix_population

# import sample population
population = load_epydemix_population("Italy")

# simulation start and end day
start_day, end_day = 0, 150

# this is how vaccinations look like in the session state
# NEW: added name field to each vaccination campaign
vaccinations = [
  { 
    "name": "vax1",
    "start": 10,
    "end": 60,
    "coverage": 0.3,
    "effectiveness": 0.9,
    "vaccinable_compartments": ["S", "R"],
    "effective_on_compartments": ["S"],
    "target_age_groups": ["0-4", "5-19", "20-49", "50-64", "65+"]
  },
  {
    "name": "vax2",
    "start": 30,
    "end": 90,
    "coverage": 0.2,
    "effectiveness": 0.8,
    "vaccinable_compartments": ["S", "E", "R"],
    "effective_on_compartments": ["S", "E"],
    "target_age_groups": ["50-64", "65+"]
  }
]

In [12]:
# get population dict for easier indexing
population_dict = {name: val for name, val in zip(population.Nk_names, population.Nk)}

def get_vaccination_doses(vx, population_dict, age_grp, start_day, end_day):
    """Compute the daily vaccination doses for a given vaccination campaign and age group."""
    # compute daily doses 
    if age_grp not in vx["target_age_groups"]:
        doses = np.zeros_like(range(start_day, end_day + 1))
    else: 
        # compute total effective doses: Nk * coverage * effectiveness
        total_eff_doses = int(vx["coverage"] * population_dict[age_grp] * vx["effectiveness"])
        # distribute doses over the campaign period
        doses = np.zeros_like(range(start_day, end_day + 1))
        doses[vx["start"]:vx["end"]] = total_eff_doses / (vx["end"] - vx["start"] + 1)
    return doses.astype(int)

In [ ]:
df_vaccination_schedule = pd.DataFrame()
# compute vaccination schedule for each campaign
for vx in vaccinations:
    vx_dict = {name: [] for name in population.Nk_names}
    vx_dict["t"] = range(start_day, end_day + 1)
    vx_dict["name"] = [vx["name"] for _ in  range(start_day, end_day + 1)]
    
    for age_grp in population.Nk_names:
        vx_dict[age_grp] = get_vaccination_doses(vx, population_dict, age_grp, start_day, end_day)
    df_vaccination_schedule = pd.concat([df_vaccination_schedule, pd.DataFrame(vx_dict)])

df_vaccination_schedule.head()

,0-4,5-19,20-49,50-64,65+,t,name
0,0,0,0,0,0,0,vax1
1,0,0,0,0,0,1,vax1
2,0,0,0,0,0,2,vax1
3,0,0,0,0,0,3,vax1
4,0,0,0,0,0,4,vax1


In [15]:
df_vaccination_schedule["t"].max()

np.int64(150)

In [9]:
# compute total effective doses: Nk * coverage * effectiveness
total_eff_doses = int(vx["coverage"] * population_dict[age_grp] * vx["effectiveness"])
# distribute doses over the campaign period
doses = np.zeros_like(range(start_day, end_day + 1))
doses[vx["start"]:vx["end"]] = total_eff_doses / (vx["end"] - vx["start"] + 1)

doses[vx["start"]:vx["end"]] = total_eff_doses / (vx["end"] - vx["start"] + 1)

In [11]:
doses.shape

(151,)